In [10]:
import nltk  # Natural Language Toolkit contient un ensemble d'outils pour nettoyer le texte et le préparer pour les modèles d'IA
nltk.download('punkt')  # Punkt is a  Sentence Tokenizer
nltk.download('wordnet') #corpus / lexical database
from nltk.stem import WordNetLemmatizer  # for lemmatization 
lemmatizer = WordNetLemmatizer()
import json  # Encodage et décodage des fichier JSON
import pickle # la sérialisation et la désérialisation d'un objet Python

import numpy as np # pour manipuler des matrices ou tableaux multidimensionnels
#pour convevoir un simple modèle on utilisé :
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout
import random
from keras.models import load_model

import tkinter # pour créer des interfaces 
from tkinter import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\samet\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
words=[] # list de tout les mots existants dans le fichier json
classes = [] # les classe = nombre des intentions 
documents = [] # chaque document est le couple : pattern et tag
useless_words = ['?', '!'] # on doit ignorer ces symboles inutile 
data = open('intents.json').read() # lecture de fichier json qui contient liste des intentions 
intents = json.loads(data)


In [12]:
intents # just pour voir le contenue de fichier json

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'tasks',
   'patterns': ['What can you do?',
    'What are your features?',
    'What are you abilities',
    'can you sing',
    'can you talk'],
   'responses': ['I can do whatever you asks me to do',
    'I can talk and do things for you',
    "Right now i'm in developing stage as soon i'm developed, I can do everything"]},
  {'tag': 'talk',
   'patterns': ['can you sing', 'can you talk', 'can you speak'],
  

In [13]:
          
#une boucle imbriquée pour extraire tous les mots des "patterns" 
#et ajouter  chacune à notre liste "words".
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        word = nltk.word_tokenize(pattern) # on doit tokenizer chaque mot
        words.extend(word) # ajout du mot à la list  Words
        documents.append((word, intent['tag'])) # ajouter un nouveau document qui correspond au couple   patterns , tag 

        
        if intent['tag'] not in classes: # ajouter une nouvelle classe s'il n'existe déja pas
            classes.append(intent['tag'])

In [14]:
#lemmatiser signifie transformer un mot en sa signification de base, ou son lemme
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in useless_words] 
words = sorted(list(set(words))) #une simple trie des mots

classes = sorted(list(set(classes)))  #une simple trie des mots

pickle.dump(classes,open('classes.pkl','wb'))  #serialiser la liste des classes
pickle.dump(words,open('words.pkl','wb')) #serialiser la liste des mots

In [15]:
training_data = []
output_empty = [0] * len(classes)
for document in documents:
    bag = [] # pour chaque document on initialize the bag of words
    pattern_words = document[0] # on prend les mots tokenizé de pattern
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words] # lemmatize chaque mot du pattern 
    for w in words: # créer un bag of word en marquant 1 si le mot existe dans le pattern sinon 0 
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1 # la class est 1 si la class correspond au tag de pattern actuel sinon 0

    training_data.append([bag, output_row])
random.shuffle(training_data) # shuffle : réorganiser l'ordre des éléments d'une liste de manière aléatoire
training_data = np.array(training_data)
train_x = list(training_data[:,0])
train_y = list(training_data[:,1])




In [7]:
# Create model - 3 layers. First layer  second layer  and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
# créer un simple modèle avec sequential  
# 3 couche dense 
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu')) #128 neurons,
model.add(Dropout(0.5)) #pour eviter le over fitting 
model.add(Dense(64, activation='relu')) #64 neurons
model.add(Dropout(0.5)) #pour eviter le over fitting
model.add(Dense(len(train_y[0]), activation='softmax'))  # softmax pour retourner la probabilité pour chaque classe

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)# Stochastic gradient descent
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy']) # categorical_crossentropy pour quantifier l'erreur de modèle deep learning  

history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1) # entrainement

model.save('chatbot_model.h5', history) #save the model


Epoch 1/200
16/16 [==============================] - 0s 1ms/step - loss: 2.9885 - accuracy: 0.0256
Epoch 2/200
16/16 [==============================] - 0s 1ms/step - loss: 2.9336 - accuracy: 0.0641
Epoch 3/200
16/16 [==============================] - 0s 1ms/step - loss: 2.8037 - accuracy: 0.1154
Epoch 4/200
16/16 [==============================] - 0s 2ms/step - loss: 2.7764 - accuracy: 0.1538
Epoch 5/200
16/16 [==============================] - 0s 1ms/step - loss: 2.7187 - accuracy: 0.1667
Epoch 6/200
16/16 [==============================] - 0s 1ms/step - loss: 2.6004 - accuracy: 0.2308
Epoch 7/200
16/16 [==============================] - 0s 1ms/step - loss: 2.4854 - accuracy: 0.2949
Epoch 8/200
16/16 [==============================] - 0s 1ms/step - loss: 2.4505 - accuracy: 0.2692
Epoch 9/200
16/16 [==============================] - 0s 1ms/step - loss: 2.3587 - accuracy: 0.3077
Epoch 10/200
16/16 [==============================] - 0s 1ms/step - loss: 2.1746 - accuracy: 0.3590
Epoch 11/

In [16]:
# aprés avoir entrainer le modele, on passe à la partie de prédiction 
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [8]:
#clean : permet de tokenize une phrase et la lemmatizer
def clean(sentence): 
    words_sentence = nltk.word_tokenize(sentence)
    words_sentence = [lemmatizer.lemmatize(word.lower()) for word in words_sentence]
    return words_sentence


#bag_words : retourner un bag of words d'une phrase entrée dans le chat 
def bag_words(sentence, words):
    
    words_sentence = clean(sentence) # clean la phrase entrée : tokenizer et lemmatizer
    bag = [0]*len(words) # initalisation de bag of words : matrice de n 0 , n = nombre de mots total
    
    for ws in words_sentence:
        for i,w in enumerate(words):
            if w == ws:
                bag[i] = 1 # assign 1 si le mot de la phrase = au mot actuelle  dans la liste de tous les mots
                
    return(np.array(bag))

def prediction(sentence, model):
    # filter out predictions below a threshold
    p = bag_words(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def final_response(classes_predicted, all_intents):
    tag = classes_predicted[0]['intent'] #prendre la classe ayant une plus haute probabilté 
    list_intents = all_intents['intents']
    for i in list_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])  # choix de reponse parmis les responses  correspondant à ce TAG
            break
    return result

def chatbot_output(message):
    classes_predicted = prediction(message, model) #prédire les classes 
    res = final_response(classes_predicted, intents) 
    return res




In [9]:
#GUI with tkinter

def send():
    message = box.get("1.0",'end-1c').strip()
    box.delete("0.0",END)

    if message != '':
        chat.config(state=NORMAL)
        chat.insert(END, "You: " + message + '\n\n')
        chat.config(foreground="#442065", font=("Verdana", 12 ))

        res = chatbot_output(message)
        chat.insert(END, "Bot: " + res + '\n\n')

        chat.config(state=DISABLED)
        chat.yview(END)


base = Tk()
base.title("Mariem BOT")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
chat = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

chat.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=chat.yview, cursor="heart")
chat['yscrollcommand'] = scrollbar.set

#Create Button to send message
button = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#18809F", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
box = Text(base, bd=0, bg="#96DCF0",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
chat.place(x=6,y=6, height=386, width=370)
box.place(x=128, y=401, height=90, width=265)
button.place(x=6, y=401, height=90)

base.mainloop()
